In [14]:
import argparse
import concurrent.futures
import logging
import os

import numpy as np
import pandas as pd
import torch
from einops import rearrange
from transformers import AutoModelForCausalLM

# python rolling_window.py --path "../../data/ES=F.csv" --input_column "Close_denoised_standardized" --output_column "Close" --prediction_length 12 --context_length 384 --frequency "H" --utc True --output "es_future_final_moirai_test.csv"

model_timemoe = AutoModelForCausalLM.from_pretrained(
    'Maple728/TimeMoE-200M',
    device_map='cpu',
    trust_remote_code=True,
)

context = pd.read_csv("../../data/ES=F.csv", parse_dates=True, index_col=0)
# select the 384 first rows
context = context["Close_denoised_standardized"].to_numpy()
context = context[:384]

prediction_length = 12


# Prepare the Time-MoE input
context_tensor = torch.tensor(context, dtype=torch.float32, device="cpu").unsqueeze(0)  # [1, context_length]

# # Forecast using Time-MoE
output_timemoe = model_timemoe.generate(context_tensor, max_new_tokens=prediction_length)
normed_predictions = output_timemoe[0][-prediction_length:].cpu().numpy().reshape(prediction_length, -1) # select only the predictions among the output
print("normed_predictions: ", normed_predictions)

print("2: ", normed_predictions[:, 0])

print("median: ", np.round(np.median(normed_predictions[:, 0], axis=0), decimals=4))


/Users/baudoincoispeau/Documents/Keensight/TF4FIP/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


normed_predictions:  [[-1.5601562]
 [-1.5584261]
 [-1.557881 ]
 [-1.5616874]
 [-1.5591675]
 [-1.5564818]
 [-1.5553168]
 [-1.5532796]
 [-1.5854827]
 [-1.5971079]
 [-1.6087503]
 [-1.6223552]]
2:  [-1.5601562 -1.5584261 -1.557881  -1.5616874 -1.5591675 -1.5564818
 -1.5553168 -1.5532796 -1.5854827 -1.5971079 -1.6087503 -1.6223552]
median:  -1.5597


In [3]:
import torch
from transformers import AutoModelForCausalLM

context_length = 12
normed_seqs = torch.randn(2, context_length)  # tensor shape is [batch_size, context_length]

model = AutoModelForCausalLM.from_pretrained(
    "Maple728/TimeMoE-200M",
    device_map="cpu",
    trust_remote_code=True,
)

# forecast
prediction_length = 6
output = model.generate(normed_seqs, max_new_tokens=prediction_length)  # shape is [batch_size, 12 + 6]
normed_predictions = output[:, -prediction_length:]  # shape is [batch_size, 6]
print(normed_predictions)

/Users/baudoincoispeau/Documents/Keensight/TF4FIP/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor([[-0.1960, -0.2479, -0.2682, -0.2474, -0.2397, -0.2355],
        [ 0.3083,  0.0530, -0.0266,  0.0668,  0.0446,  0.0169]])
